Установим torchmetrics, чтобы руками не считать точность, и склонируем с гитхаба датасет.

In [ ]:
!pip install torchmetrics

In [ ]:
!git clone https://github.com/fortvivlan/catset.git
%cd /content/catset

Imports

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets
from torchvision.transforms import Compose, Normalize, ToTensor
from torch.utils.data import Subset
from torchmetrics import Accuracy as VAccuracy
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings('ignore')

Теперь нам нужно подготовить датасет к работе. Картинки неплохо бы нормализовать: помним, что любая картинка эссеншали только матрица с чиселками. Сделаем наши картинки не слишком яркими, чтобы все они были примерно одинаковыми по интенсивности цвета и подобное. Для этого нужно предпосчитать среднее и стандартное отклонение по датасету:

In [6]:
def get_mean_and_std(dataloader):
    '''Считаем среднее и стандартное отклонение'''
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader: # data - батч с картинками, _ - y_true, но они нам щас не нужны
        channels_sum += torch.mean(data, dim=[0, 2, 3]) # [batch_size x 3 x image_size x image_size]
        channels_squared_sum += torch.mean(data ** 2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches

    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

Заготовим трансформацию картинки: мы должны будем все картинки в батче превратить в тензоры (чтобы потом по ним рассчитать их среднее и ст. отклонение.

Торч умеет работать с датасетами прямо из папки: для картинок это очень удобно, потому что иначе вам пришлось бы загружать все свои тяжелые картинки в оперативную память, было бы медленно и тупо.

In [7]:
transform = Compose(
    [ToTensor()]
)

# загрузим просто все картинки махом (хотя вообще-то нормализовать надо только по трейну, но ладно, мы учимся)
dataset = torch.utils.data.DataLoader(datasets.ImageFolder('Cats', transform=transform), batch_size=64, shuffle=False)

In [8]:
mean, std = get_mean_and_std(dataset)
print(mean, std)

tensor([0.3851, 0.3576, 0.3296]) tensor([0.2769, 0.2711, 0.2644])


Получили нужные чиселки: теперь нужно добавить их в трансформацию, которую будем использовать уже в реальной подготовке датасета.

In [9]:
transform = Compose(
    [
        ToTensor(),
        Normalize((0.3851, 0.3576, 0.3296), (0.2769, 0.2711, 0.2644))
    ]
)
dataset = datasets.ImageFolder('Cats', transform=transform)

Напишем функцию, которая будет сплитить наш датасет: сперва отшаффлим индексы и разделим их стандартной ск-лерновской тулзой, а потом воспользуемся утилитой торча Subset.

In [10]:
def train_val_dataset(dataset, val_split=0.25):
    '''Create train and validation datasets'''
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [11]:
traintest = train_val_dataset(dataset)

In [12]:
train_dataloader = torch.utils.data.DataLoader(traintest['train'], batch_size=16, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(traintest['val'], batch_size=16, shuffle=False)

Ну вот, теперь наш датасет готов, пора писать трейнлуп и архитектурку.

In [13]:
def train(model, optimizer, n_epochs=5):
    for epoch in range(1, n_epochs + 1):
        # train
        for x_train, y_train in tqdm(train_dataloader):
            y_pred = model(x_train)
            loss = F.cross_entropy(y_pred, y_train) # используем кросс-энтропию
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # validation
        if not epoch % 2:
            val_loss = []
            val_accuracy = []
            with torch.no_grad():
                for x_val, y_val in tqdm(val_dataloader):
                    y_pred = model(x_val)
                    loss = F.cross_entropy(y_pred, y_val)
                    val_loss.append(loss.numpy())
                    val_accuracy.extend((torch.argmax(y_pred, dim=-1) == y_val).numpy().tolist())

            print(f"Epoch: {epoch}\tloss: {np.mean(val_loss)}\taccuracy: {np.mean(val_accuracy)}")

In [14]:
class Torchic(nn.Module):
    def __init__(self):
        super().__init__()
        # VGG
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5) # 3 канала, 10 ядер, размер ядра - 5
        # картинка сожмется на x - (kernel_size - 1)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        # картинка сожмется на x - (kernel_size - 1)
        self.pool = nn.MaxPool2d(2) # макс-пулинг: уменьшим на 2
        # картинка сожмется в два раза
        self.flatten = nn.Flatten()

        # head
        # исходные картинки были 256х256х3. После первого слоя сверток стало: 252х252х10)
        # после второго слоя сверток стало: 248х248х20
        # после пулинга стало 124х124х20
        self.fc1 = nn.Linear(124 * 124 * 20, 128) # слой на 128 нейронов
        self.fc2 = nn.Linear(128, 2) # выходной слой на 2 нейрона (можно было 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [15]:
torchic = Torchic()
optimizer = opt.Adam(torchic.parameters(), lr=0.001)

In [ ]:
train(torchic, optimizer, 10)

In [17]:
def matplotlib_imshow(img, one_channel=False):
    """A function for plotting unnormalized images, but it still gets clipping warning"""
    if one_channel:
        img = img.mean(dim=0)
    img = img * std + mean
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (0, 1, 2)))

In [ ]:
classes = {0: 'Лисена', 1: 'Мороша'}

torchic.eval()
for i in range(10):
    # нам здесь приходится перевернуть, чтобы обратно разнормализовать нашу картинку
    matplotlib_imshow(traintest['val'][i][0].permute(1, 2, 0))
    plt.show()
    print(f"Dat is {classes[traintest['val'][i][1]]}")
    ypred = torchic(traintest['val'][i][0].unsqueeze(1).permute(1, 0, 2, 3))
    print(f"Torchic thinks it is {classes[torch.argmax(ypred, dim=-1).item()]}")